# 3. Modelado y Clasificación de Sentimiento

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    confusion_matrix, 
    classification_report
)

Cargo el dataset que preprocesé en el notebook anterior

In [2]:
df = pd.read_csv('./dataset/Movies-and-TV-preprocessed.csv')

In [3]:
df.head()

,review,processed_review,sentiment
0,One of the other reviewers has mentioned that ...,one reviewers mentioned watching episode youll...,positive
1,A wonderful little production. <br /><br />The...,wonderful little production filming technique ...,positive
2,I thought this was a wonderful way to spend ti...,thought wonderful way spend time hot summer we...,positive
3,Basically there's a family where a little boy ...,basically theres family little boy jake thinks...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",petter matteis love time money visually stunni...,positive


Prepararo datos para entrenamiento, voy a separar los datos en conjuntos de entrenamiento 80% y test 20%.
Uso la separación 80% entrenamiento y 20% test porque me permite entrenar el modelo con suficientes datos para que aprenda bien, y al mismo tiempo reservar una parte independiente para evaluar su rendimiento.

In [4]:
X = df['processed_review']
y = df['sentiment'].map({'positive': 1, 'negative': 0})

In [5]:
print("Distribución de y:")
print(y.value_counts())

Distribución de y:
1    25000
0    25000
Name: sentiment, dtype: int64


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Entrenamiento: {len(X_train)}")
print(f"Test: {len(X_test)}")

Entrenamiento: 40000
Test: 10000


Voy a usar TfidfVectorizer para convertir el texto en vectores numéricos. 

por estas razones:
- TF-IDF da más peso a palabras importantes y menos a las muy comunes
- Los bigramas ayudan a capturar frases como "not good" que tienen sentido diferente a "good" solo
- Limitar features reduce overfitting y tiempo de entrenamiento

Parámetros elegidos:
- max_features=5000: Limito el vocabulario a las 5000 palabras más frecuentes para reducir dimensionalidad
- ngram_range=(1, 2): Uso unigramas y bigramas para capturar contexto
- min_df=5: Ignoro palabras que aparecen en menos de 5 documentos para eliminar ruido
- max_df=0.9: Ignoro palabras que aparecen en más del 90% de documentos para eliminar palabras muy comunes


In [7]:
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.9
)

In [8]:
X_train_vectorized = vectorizer.fit_transform(X_train)

print(f"Forma de X_train vectorizado: {X_train_vectorized.shape}")
print(f"Vocabulario del vectorizador: {len(vectorizer.vocabulary_)} términos")

Forma de X_train vectorizado: (40000, 5000)
Vocabulario del vectorizador: 5000 términos


In [9]:
X_test_vectorized = vectorizer.transform(X_test)

print(f"Forma de X_test vectorizado: {X_test_vectorized.shape}")

Forma de X_test vectorizado: (10000, 5000)


## Modelo 1: Naive Bayes Multinomial

Empiezo con Naive Bayes porque es rápido y funciona bien con bag-of-words para clasificación de texto.

In [10]:
nb_model = MultinomialNB(alpha=1.0)

Entrenamiento del modelo

In [11]:
nb_model.fit(X_train_vectorized, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


Hago las predicciones

In [12]:
nb_train_pred = nb_model.predict(X_train_vectorized)
nb_test_pred = nb_model.predict(X_test_vectorized)
nb_test_proba = nb_model.predict_proba(X_test_vectorized)[:, 1]

## Modelo 2: Support Vector Machine - SVM

Como segundo modelo utilizo SVM lineal porque es especialmente eficaz en problemas de clasificación de texto, donde los datos suelen ser de alta dimensionalidad y esparcidos

In [13]:
svm_model = LinearSVC(C=1.0, max_iter=1000, random_state=42)

Entreno el modelo SVM

In [14]:
svm_model.fit(X_train_vectorized, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,42


In [15]:
svm_train_pred = svm_model.predict(X_train_vectorized)
svm_test_pred = svm_model.predict(X_test_vectorized)

## Evaluación de modelos

Voy a calcular varias métricas para comparar los modelos.

In [16]:
def evaluar_modelo(y_true, y_pred, nombre_modelo):
    print(f"Resultados para: {nombre_modelo}")
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"Métricas:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nMatriz de Confusión:")
    print(cm)
    
    print(f"\nReporte de Clasificación:")
    print(classification_report(y_true, y_pred, target_names=['Negative', 'Positive']))
    
    return accuracy, precision, recall, f1

Evaluación en conjunto de entrenamiento

In [17]:

nb_train_metrics = evaluar_modelo(y_train, nb_train_pred, "Naive Bayes (Train)")

Resultados para: Naive Bayes (Train)
Métricas:
Accuracy:  0.8673
Precision: 0.8577
Recall:    0.8809
F1-Score:  0.8691

Matriz de Confusión:
[[17076  2924]
 [ 2383 17617]]

Reporte de Clasificación:
              precision    recall  f1-score   support

    Negative       0.88      0.85      0.87     20000
    Positive       0.86      0.88      0.87     20000

    accuracy                           0.87     40000
   macro avg       0.87      0.87      0.87     40000
weighted avg       0.87      0.87      0.87     40000



In [18]:
svm_train_metrics = evaluar_modelo(y_train, svm_train_pred, "SVM (Train)")

Resultados para: SVM (Train)
Métricas:
Accuracy:  0.9300
Precision: 0.9260
Recall:    0.9348
F1-Score:  0.9304

Matriz de Confusión:
[[18505  1495]
 [ 1304 18696]]

Reporte de Clasificación:
              precision    recall  f1-score   support

    Negative       0.93      0.93      0.93     20000
    Positive       0.93      0.93      0.93     20000

    accuracy                           0.93     40000
   macro avg       0.93      0.93      0.93     40000
weighted avg       0.93      0.93      0.93     40000



### Evaluación en conjunto de test

In [19]:
nb_test_metrics = evaluar_modelo(y_test, nb_test_pred, "Naive Bayes (Test)")

Resultados para: Naive Bayes (Test)
Métricas:
Accuracy:  0.8592
Precision: 0.8474
Recall:    0.8762
F1-Score:  0.8616

Matriz de Confusión:
[[4211  789]
 [ 619 4381]]

Reporte de Clasificación:
              precision    recall  f1-score   support

    Negative       0.87      0.84      0.86      5000
    Positive       0.85      0.88      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [20]:
svm_test_metrics = evaluar_modelo(y_test, svm_test_pred, "SVM (Test)")

Resultados para: SVM (Test)
Métricas:
Accuracy:  0.8836
Precision: 0.8815
Recall:    0.8864
F1-Score:  0.8839

Matriz de Confusión:
[[4404  596]
 [ 568 4432]]

Reporte de Clasificación:
              precision    recall  f1-score   support

    Negative       0.89      0.88      0.88      5000
    Positive       0.88      0.89      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



## Comparación visual de modelos

In [21]:
metricas_comparacion = pd.DataFrame({
    'Modelo': ['Naive Bayes', 'SVM'],
    'Accuracy': [nb_test_metrics[0], svm_test_metrics[0]],
    'Precision': [nb_test_metrics[1], svm_test_metrics[1]],
    'Recall': [nb_test_metrics[2], svm_test_metrics[2]],
    'F1-Score': [nb_test_metrics[3], svm_test_metrics[3]]
})

print("Comparación de métricas en Test:")
print(metricas_comparacion)

Comparación de métricas en Test:
        Modelo  Accuracy  Precision  Recall  F1-Score
0  Naive Bayes    0.8592   0.847389  0.8762  0.861554
1          SVM    0.8836   0.881464  0.8864  0.883925


Por ultimo guardo las predicciones

In [22]:
predictions_df = pd.DataFrame({
    'true_label': y_test,
    'nb_prediction': nb_test_pred,
    'nb_probability': nb_test_proba,
    'svm_prediction': svm_test_pred
})

predictions_df.to_csv('./dataset/predictions.csv', index=False)


## Conclusion

Después de entrenar y evaluar dos modelos de clasificación de sentimiento, puedo concluir lo siguiente:

Naive Bayes Multinomial:
- Modelo probabilístico simple y rápido
- Asume independencia entre características simplificación fuerte
- Funciona bien con bag-of-words
- Tiempo de entrenamiento muy rápido

Support Vector Machine:
- Modelo más robusto que busca el hiperplano óptimo
- No asume independencia de características
- Suele generalizar mejor
- Tiempo de entrenamiento un poco mayor

Basándome en las métricas calculadas:

1. Accuracy: Ambos modelos tienen accuracy similar, alrededor del 85-90%
2. Precision: Indica qué tan confiables son las predicciones positivas
3. Recall: Indica qué proporción de positivos reales se detectaron
4. F1-Score: Balance entre precision y recall

Selecciono el SVM como mejor modelo porque:
- Tiene métricas ligeramente superiores en general
- Menor tasa de falsos positivos
- Mejor generalización en datos no vistos
- Más robusto ante variaciones en los datos

2. Vectorización: Usé TF-IDF con n-gramas (1,2) porque:
   - Captura mejor la importancia de las palabras
   - Los bigramas ayudan con negaciones ("not good")
   - Reducción de dimensionalidad controlada (5000 features)